<center>
<img src="https://laelgelcpublic.s3.sa-east-1.amazonaws.com/lael_50_years_narrow_white.png.no_years.400px_96dpi.png" width="300" alt="LAEL 50 years logo">
<h3>APPLIED LINGUISTICS GRADUATE PROGRAMME (LAEL)</h3>
</center>
<hr>

# Images function set - Troubleshooting for Yara
The purpose of this document is to register the findings of the troubleshooting process for Yara. The approach consists of reproducing the outcomes on a separate environment and comparing with the ones obtained by Yara.

Please refer to the following repository:
- [cl_images_yara](https://github.com/laelgelc/cl_images_yara)

## Functions 'presample' to 'collecturls'

- The same 'urls.txt' that Yara obtained could be reproduced - they are identical

<center>
<img src="https://laelgelcclimages.s3.sa-east-1.amazonaws.com/Figure-20231118-073801.png" width="800" alt="Figure-20231118-073801.png">
</center>

- So far, the 'images_index.txt' that was obtained has a number of lines that matches the index: 040065
- It contains more lines than Yara's one, which means that further processing in other functions will remove images that are not suitable such as duplicates and empty files
- 50 folders to store the images were created as expected

<center>
<img src="https://laelgelcclimages.s3.sa-east-1.amazonaws.com/Figure-20231118-074532.png" width="800" alt="Figure-20231118-073801.png">
</center>

## Function 'grabimages'

- Inspecting the images, there are empty files. They are supposed to be detected and removed from 'images_index.txt'

<center>
<img src="https://laelgelcclimages.s3.sa-east-1.amazonaws.com/Figure-20231118-082231.png" width="800" alt="Figure-20231118-073801.png">
</center>

- While running 'grabimages' the performance of the server degraded gradually to the point of becoming no longer reachable. Eventually, it had to be rebooted. The image collection stopped at image 032258 after a series of empty files which should not have been empty

<center>
<img src="https://laelgelcclimages.s3.sa-east-1.amazonaws.com/Figure-20231119-1022.jpeg" width="800" alt="Figure-20231118-073801.png">
</center>

- The image 032258.jpg actually exists and could have been collected
  - fl:33|t:032258|id:1553193928903508000|d:2022-07-30|u:cicero78713658|i:https://pbs.twimg.com/media/FY4NpX8WAAEh5KM?format=jpg&name=large|f:jpg

<center>
<img src="https://laelgelcclimages.s3.sa-east-1.amazonaws.com/Figure-20231119-1033.jpeg" width="300" alt="Figure-20231118-073801.png">
</center>

- The image 031983.jpg Yara reported as corrupted could be successfully collected this time

<center>
<img src="https://laelgelcclimages.s3.sa-east-1.amazonaws.com/Figure-20231119-1040.jpg" width="300" alt="Figure-20231118-073801.png">
</center>

- The evidence shows that 'grabimages' or something in its context is not reliable. The function is going to be re-run in an attempt to obtain better results.

<center>
<img src="https://laelgelcclimages.s3.sa-east-1.amazonaws.com/Figure-20231119-163047.png" width="800" alt="Figure-20231118-073801.png">
</center>

- The server became unreachable again and had to be rebooted again. It has been determined that the issue happened after the collection of image 035225.jpg. Images 035226.jpg and 035227.jpg are not truly empty. This time, the 'grabimages' function will be altered in order to collect images from 035226.jpg on
  - fl:36|t:035226|id:1586858281166856200|d:2022-10-30|u:myaorthammy|i:https://pbs.twimg.com/media/FgWnMlPXoAEPCFQ?format=jpg&name=large|f:jpg
- The process completed apparently successfully but the process as a whole **could not be considered reliable**. Therefore, **the function 'grabimages' was run directly as a shell script without being mediated by a Python programme. This time, the process was completed smoothly and reliably**

## Function 'removedupes'

### Determining the empty images

- The empty images were determined by running the following command
  - A list of 1954 empty images was obtained

In [ ]:
(my_env) ubuntu@ip-172-31-46-151:~/my_env/cl_images$ find images/images -type f -empty > emptyfiles.txt

- Before running 'removedupes', the file 'images_index.txt' had 40065 lines with the image index matching the number of the corresponding line

- After running 'removedupes':
  - the function did not find duplicated posts because the auxiliary file 'd' was empty
  - all of the empty images were removed
  - the file 'images_index.txt' had 38459 lines (the image index no longer matching the number of the corresponding line). Therefore, (40065 - 38459 = 1606) images have been removed. That means that 12 files other than the empty ones were removed - this is the case of images 000463 and 001030. Both of them have an image link in the field 'f'
    - fl:01|t:000463|id:1099442009041965000|d:2019-02-23|u:jmb_newage|i:https://pbs.twimg.com/ext_tw_video_thumb/1099395251339444224/pu/img/Wiuwv4JK0jWvPBCm.jpg|f:https://pbs.twimg.com/ext_tw_video_thumb/1099395251339444224/pu/img/Wiuwv4JK0jWvPBCm.jpg
    - fl:02|t:001030|id:1121917120416223200|d:2019-04-26|u:mauroba15398952|i:https://pbs.twimg.com/media/D5Gg0EmXsAMmdNe.jpg|f:https://pbs.twimg.com/media/D5Gg0EmXsAMmdNe.jpg

<center>
<img src="https://laelgelcclimages.s3.sa-east-1.amazonaws.com/Figure-20231120-081956.png" width="800" alt="Figure-20231118-073801.png">
</center>

- The following command was used to create a list of images that 'removedupes' removed, 'images_index_before_after_rogerio_diff.txt'

In [ ]:
(my_env) ubuntu@ip-172-31-46-151:~/my_env/cl_images_yara$ diff images_index_before_dedupe.txt images_index_after_dedupe_rogerio.txt | grep fl: > images_index_before_after_rogerio_diff.txt

- Comparing with the results obtained by Yara, her 'images_index_after_dedupe_yara.txt' contains 38603. That means that she obtained (38603 - 38459 = 144) more images. One possible explanation is that the Twitter users removed a few posts since Yara collected the images

## Function 'uploadtobucket'

- The function completed successfully
- The code required a slight adaptation to run over Ubuntu

In [ ]:
(my_env) ubuntu@ip-172-31-46-151:~/my_env/cl_images$ more nohup.out
Directory ./images/images uploaded to laelimages/images successfully!
(my_env) ubuntu@ip-172-31-46-151:~/my_env/cl_images$ 

## Function 'googlelabels'

### Exception handling

- An exception was raised while processing the image 008136.jpg, located in the row i=7926:

---- detect-labels 7923 / 40065 ----

---- detect-labels 7924 / 40065 ----

---- detect-labels 7925 / 40065 ----

---- detect-labels 7926 / 40065 ----

Traceback (most recent call last):

  File "/home/ubuntu/my_env/cl_images/climages.py", line 93, in <module>
    
    labels = detect_labels(image_uri)
    
  File "/home/ubuntu/my_env/cl_images/climages.py", line 58, in detect_labels
    
    raise Exception(f'Error: {response.error.message}')
    
Exception: Error: Internal server error. Failed to process features.

- There was nothing wrong with image 008136.jpg
- The function 'googlelabels' was suspended due to an exception reported by the Google Vision service. The service experienced an internal error and failed. It means that the error occurred at Google Vision's side
- In order to resume the processing from the point it stopped, the server was rebooted just to get it freshly started and the function 'googlelabels' was adjusted as follows:

for i in range(7926, last + 1):
    
    try:
    
        with open('images/images_index.txt', 'r', encoding='utf8') as f:
    
            line = next(line for j, line in enumerate(f, start = 1) if j == i)
    
            folder = line.split('|')[0][3:]
    
            n = line.split('|')[1][2:]
    
            id = line.split('|')[2][3:]
    
            file = line.split('|')[5][2:]
    
            ext = line.split('|')[6][2:5]
    

        print(f"---- detect-labels {i} / {last} ----")

- The processing was resumed successfully

#### Takeaways

- This exception shed light on the logic of the function 'googlelabels'
  - The counter variable 'i' holds the row of the file 'images_index.txt'. Since the function 'removedupes' removed a few images, the number of the rows no longer matches the index of the images. This explains the discrepancy between the counter 'i' and the name of the image
  - The variable 'last', on the other hand, holds the index of the last row of the file 'images_index.txt'. In our case, it is set to 40065. It means that the processing of labels will try to iterate beyond the quantity of images to be processed because now 'images_index.txt' contains 38459 images. Therefore, when the function 'googlelabels' tries to process the row 38460, it will not find any values and the programme will break
- In spite of this, by the time the programme reaches this breaking point, its purpose will have been served. Therefore, we will manage this as an exception and live with the problem

## Lessons learned

Yara's project was the first opportunity to employ Jupyter Notebooks to process Corpus Linguistics research data. Here are a few reflections on the lived experience

1. The issues faced by Yara were probably due to the limited performance of a personal computer and home broadband access in the face of heavy workloads. When bottlenecks hit a personal computer, files may end up corrupted
2. While Jupyter Notebooks are highly didactic, its processing relies on a Python kernel whose performance cannot be compared to a Python interpreter running over a Linux server. Jupyter Notebooks are ideal for testing ideas and prototyping solutions, they may not be so when it comes to dealing with heavy workloads
3. Cloud Computing is probably a better solution for GELC's research projects due to following reasons:
- The projects pay only for the resources the utilise. There is no need to source and maintain hardware
- The computing service is provided by server-grade virtual machines that are dedicated and tuned to the required workload
- It provides easy-to-deploy infrastructure to process Big Data workloads

The following figure shows the monitoring of the server of this project when it was running the 'googlelabels' function. The CPU utilisation remained steady at 10%.

<center>
<img src="https://laelgelcclimages.s3.sa-east-1.amazonaws.com/Figure-20231120-1623.jpg" width="800" alt="Figure-20231118-073801.png">
</center>